In [1]:
import numpy as np
import pandas as pd
import time
import gensim
import re
from gensim.models.word2vec import Word2Vec 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('../adam/datasets/scrape_5.11.csv')

pd.set_option('display.max_colwidth', -1)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start
0,1228415289269858310,Bearjew1964,"Disease modelers gaze into their computers to see the future of Covid-19, and it isn’t good https://www.statnews.com/2020/02/14/disease-modelers-see-future-of-covid-19/ … via @statnews",2020-02-14 20:26:42,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
1,1228345808954765313,Carlos Salazar,"I know we have a limited attention span and all, but this might be way more devastating than Covid-19. https://twitter.com/BBCWorld/status/1228290268857802752 …",2020-02-14 15:50:37,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
2,1228149699145871360,Hefe O-Ren Ishii,"As one Princess cruise is quarantined for COVID-19, another is hit with different illness https://www.nbcnews.com/news/us-news/one-princess-cruise-quarantined-covid-19-another-hit-different-illness-n1136436 … via @nbcnews",2020-02-14 02:51:20,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
3,1228008158586720256,josh,"how about we call it by it’s new name: COVID-19. and whats with the dramatic ass imagery? quit fear mongering, thanks",2020-02-13 17:28:54,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
4,1227684262859636738,Helen Ong,"Scorpio: To me, the #COVID_19 19, as virulent as it appears to be, is clearly containable if NYC has yet to see a confirmed case of infection. \n\nThe City is virtually a microcosm of the world, in numbers as well as in diversity. \n\n#CoronaVirus\n#God_bless_you_Wuhan!",2020-02-12 20:01:52,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(96507, 10)

In [5]:
df['token_text'] = df['text'].str.replace('([^ a-zA-Z0-9])', '')
df['split_text'] = df['token_text'].astype(str).str.lower().str.split()

In [6]:
sent = [row for row in df['split_text']]

In [70]:
tweet_w2v = Word2Vec(size=2000, min_count=1000, window=100)

In [71]:
tweet_w2v.build_vocab(sent)

In [74]:
tweet_w2v.most_similar('test')

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('manhattan', 0.06396234035491943),
 ('going', 0.06152564287185669),
 ('positive', 0.05483439564704895),
 ('very', 0.050796493887901306),
 ('weeks', 0.05075221508741379),
 ('great', 0.04883856326341629),
 ('even', 0.04624320566654205),
 ('a', 0.04469980299472809),
 ('gloves', 0.0443817563354969),
 ('without', 0.039375271648168564)]